# Scraping Startup Data from NIC (National Incubation Center) Hyderabad Sindh, Pakistan

## Introduction

The National Incubation Center (NIC) Hyderabad is a hub for fostering innovation and entrepreneurship in Sindh, Pakistan. This notebook aims to scrape and compile data on startups from the NIC Hyderabad website. The objective is to gather information on the startups, including their names, descriptions, and links to their detailed profiles, which can be used for further analysis and research.

## 1. Setup and Libraries

In this section, we will import the necessary libraries required for web scraping and data processing.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup

## 3. Web Scraping
### Base URL and Headers
We define the base URL for NIC Hyderabad cohorts and set headers to mimic a real browser.

In [2]:
# Base URL adding 1, 2, 3, end of will make the change the round/cohort
url = 'https://nichyderabad.com/about/cohort-'

# Set headers to mimic a real browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

## 3. Fetching Data from NIC Hyderabad
We fetch the web pages for each cohort and parse the HTML content.

In [3]:
# to temprary save all the responces
responses = {'cohorts':[], 'sope':[]}

# itrating ovwer all the cohorts page
for i in range(1, 5): # till this date 24/06/2024 we only have four cohorts
  # fatching the page of the iterate index
  response= requests.get(url+f'{i}', headers=headers)
  # only if the request is seccefull
  if response.status_code == 200:
    # save the cohort to the responses
    responses['cohorts'].append(i)
    # turning the responce to sope
    soup = BeautifulSoup(response.content, 'html.parser')
    # saving that sope in responses
    responses['sope'].append(soup)
  else: print(response.status_code, 'for searching cohort', i)

### Extracting Data for Each Cohort
We extract the relevant data (company names, URL of the each startup's profile page, and cohort session) from each cohort's page.
#### Cohort 1

In [4]:
# for cohort 1
soup_cohort1 = responses['sope'][0]

In [5]:
# selecting all "div" eiliments with class content-container
content_container = soup_cohort1.find_all('div', class_='content-container')
print('Total number of retrieve compony in cohort-1', len(content_container))

Total number of retrieve compony in cohort-1 20


In [6]:
NIC = {'NIC-URL':[], 'cohort':[]}

for i in content_container:
  if i.find('p').find_all('a') != []:
    startup_url=i.find('p').find_all('a')[0].get('href')
    NIC["NIC-URL"].append(startup_url)

# adding 1 to each cohort equel to the number of the other observation
NIC["cohort"]=[1]*len(NIC["NIC-URL"])

In [7]:
NIC_df = pd.DataFrame(NIC)
NIC_df.head(20)

,NIC-URL,cohort
0,https://nichyderabad.com/about/cohort-1/emplai/,1
1,https://nichyderabad.com/about/cohort-1/agridu...,1
2,https://nichyderabad.com/about/cohort-1/seevit...,1
3,https://nichyderabad.com/about/cohort-1/monitr/,1
4,https://nichyderabad.com/about/cohort-1/upni-m...,1
5,https://nichyderabad.com/about/cohort-1/techfu...,1
6,https://nichyderabad.com/about/cohort-1/pak-bi...,1
7,https://nichyderabad.com/about/cohort-1/qualit...,1
8,https://nichyderabad.com/about/cohort-1/rentary/,1
9,https://nichyderabad.com/about/cohort-1/blocks...,1


#### Fetching the all the other cohorts

In [8]:
total_page = len(responses['sope'])
# cohort 2 and on
for i in range(1, total_page):
  sope_cohort = responses['sope'][i]
  # find all "a" tages with heading-link class ans store in list
  a_tags = sope_cohort.find_all('a', class_='heading-link')
  print(f'Total number of retrieve compony in cohort-{i+1} are', len(a_tags))
  for j in a_tags:
    # add the company, url, and cohort 2 to the NIC_df
    NIC_df.loc[len(NIC_df)] = [j.get('href'), 2]

Total number of retrieve compony in cohort-2 are 28
Total number of retrieve compony in cohort-3 are 20
Total number of retrieve compony in cohort-4 are 14


In [9]:
NIC_df.drop_duplicates(subset='NIC-URL', inplace=True)

In [10]:
NIC_df[NIC_df['NIC-URL'].duplicated(keep=False)]

,NIC-URL,cohort


*Note: Now that we have the like for each page of the startup on NIC website, we will start to fatch the data from thos page*

In [11]:
def get_soup(link_:str) -> BeautifulSoup:
    response = requests.get(link_, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    else:
        print(response.status_code, 'for', link_)

In [12]:
import re

for i in NIC_df['NIC-URL'].index:
  soup = get_soup(NIC_df['NIC-URL'][i])
  table_div = soup.find('div', class_= re.compile(r'fusion-text fusion-text-2.*'))
  print(i, NIC_df['NIC-URL'][i])
  table = table_div.find('table')

  # Iterate over each row in the table
  for row in table.find_all('tr'):
      # Extract columns from each row
      columns = row.find_all('td')
      # Get the text content of each column and strip any extra whitespace
      columns = [col.get_text(strip=True) for col in columns]
      # add this info to the NIC_df where only have three columns [company	NIC-URL	cohort]
      if len(columns) >= 2:
        NIC_df.loc[i, columns[0]] = columns[1]

0 https://nichyderabad.com/about/cohort-1/emplai/
1 https://nichyderabad.com/about/cohort-1/agridunya-technologies/
2 https://nichyderabad.com/about/cohort-1/seevitals-solutions/
3 https://nichyderabad.com/about/cohort-1/monitr/
4 https://nichyderabad.com/about/cohort-1/upni-market/
5 https://nichyderabad.com/about/cohort-1/techfuturic/
6 https://nichyderabad.com/about/cohort-1/pak-bioenergy/
7 https://nichyderabad.com/about/cohort-1/quality-butler-services/
8 https://nichyderabad.com/about/cohort-1/rentary/
9 https://nichyderabad.com/about/cohort-1/blockshippro/
10 https://nichyderabad.com/about/cohort-1/salon-tracer/
11 https://nichyderabad.com/about/cohort-1/nanotechx/
12 https://nichyderabad.com/about/cohort-1/dbargain/
13 https://nichyderabad.com/about/cohort-1/sahulat/
14 https://nichyderabad.com/about/cohort-1/gems-rock-bid/
15 https://nichyderabad.com/about/cohort-1/verbue/
16 https://nichyderabad.com/about/cohort-1/mixeal/
17 https://nichyderabad.com/about/cohort-1/alt-ed/
18 

In [13]:
NIC_df.columns

Index(['NIC-URL', 'cohort', 'STARTUP', 'STARTUP STAGE', 'DOMAIN', 'WEBSITE',
       'FOUNDER', 'CONTACT', 'EMAIL'],
      dtype='object')

In [14]:
NIC_df = NIC_df.replace('', pd.NA).replace('–', pd.NA)

## 4. Saving the Data
Finally, we save the collected and processed data to a CSV file for future analysis.

In [15]:
NIC_df.to_csv('NIC-hydrabad_startups-data.csv')

## Accessing the Data

The dataset generated from this notebook can be accessed through GitHub: [NIC Startups Data](https://github.com/codewithawr/NIC-hydrabad-Startups-Data)

## Appendix
### Additional Information
- Source: National Incubation Center (NIC) Hyderabad website
- Date of Data Collection: 24/06/2024
- Acknowledgments: Thanks to NIC Hyderabad for providing access to the startup information